In [13]:
import numpy;
import math;
import matplotlib.pyplot as plt

plot = False

def getData(N, variance):
    """
    Generates a dataset {(xi, yi) : i = 1, 2, . . .N} of N (X, Y ) pairs for a given value of N and sigma^2
    :return: The generated dataset of (X,Y) pairs
    """
    X = numpy.random.rand(N) # N values between 0 and 1
    Z = numpy.random.normal(0, math.sqrt(variance), N) # Zero mean Gaussian random variable, with standard deviation, of length N
    Y = numpy.cos(2*math.pi*X) + Z # Y generated from function (1)
    #print("X: ", X)
    #print("Z: ", Z)
    #print ("Y: ", Y)

    # store the (X,Y) pairs in a N x 2 matrix
    x_y_pair_matrix = numpy.column_stack((X, Y))
    #print("(X,Y) pairs:", x_y_pair_matrix)
    # optionally plot the dataset that is created
    if plot:
        plt.scatter(X,Y, label="My Random Dataset", marker='x')
        plt.title('Generated Dataset for cos(2*pi*x) from 0 to 1')
    return x_y_pair_matrix

def getMSE(data_set, poly_coefficients):
    X_values = numpy.array(data_set[:, 0]) # Values of x from given dataset
    y_true_values = numpy.array(data_set[:, 1]) # Values of y from given dataset

    # Vandermonde matrix of the function, multiplied by the weights/coefficients gives y_pred
    vander_matrix = numpy.vander(X_values, len(poly_coefficients), True)
    y_pred = vander_matrix.dot(poly_coefficients) 

    # MSE calculation using numpy, but will manually calculate below
    mse = numpy.mean((y_true_values - y_pred)**2) 
    #print ("getMSE_Updated - Auto MSE:", mse)

    # Sum all the squared differences of each y_pred, y_true. Then average that over the vector length, N 
    summation_diffs_squared = 0
    for y_pred_i, y_true_i in zip(y_pred, y_true_values):
        squared_diff = (y_pred_i - y_true_i) ** 2
        summation_diffs_squared += squared_diff
    mse = summation_diffs_squared / len(y_true_values)
    #print ("getMSE_Updated - Manual MSE: ", mse)
    return mse

def getMSEOld(y_predicted, y_true): # this is probably won't be needed!!!
    """
    Get the Mean Square Error (MSE) loss from the predicted dataset, and the true dataset
    :return: The Mean Square Error (MSE)
    """
    # MSE calculation using numpy, but will manually calculate below
    mse = numpy.mean((y_true - y_predicted)**2) 
    #print ("getMSEOld - MSE:", mse)
    return mse

def fitData(data_set, degree_poly, test_dataset):
    """
    fitData method that estimates the polynomial coefficients by fitting a given data to a degree-d polynomial
    :return: estimated_coefficients, Ein (MSE), Eout
    """
    # Hyperparameters of GD
    learning_rate = 0.1
    iterations = 1000
    X_values = numpy.array(data_set[:, 0]) # Values of x from given dataset, vector is 1xN
    Y_values = numpy.array(data_set[:, 1]) # Values of y from given dataset, vector is 1xN
    N = len(Y_values)
    #print("Real X values:\n", X_values)
    #print("Real Y values:\n", Y_values)

    # Random starting coefficents
    weights = startingRandomCoeffs(degree_poly+1) # vector is 1xD
    #print("Staring Weights:\n", weights)

    X_polynomial = numpy.poly1d(weights)
    #print("Poly function auto\n", X_polynomial, "\n")

    # Vandermonde matrix, for all X values, without coefficients of N X (D+1) size
    vander_matrix = numpy.vander(X_values, degree_poly+1, True)
    #print("Vander Matrix:\n", vander_matrix)

    Y_pred = vander_matrix.dot(weights)
    for x in range(iterations):

        Y_pred = vander_matrix.dot(weights)
        grad = 2*(learning_rate/N) * ((Y_values - Y_pred)).dot(vander_matrix)
        weights += grad
        #if x % 500 == 0:
            #print("Predicted Y values:\n", Y_pred)
            #print("Current MSE:", getMSEOld(Y_pred, Y_values))
            #print("Gradients:\n", grad)

    if plot:
        plt.scatter(X_values,Y_pred, label="My Random Dataset", marker='o')
        plt.title('Predicted Y Values')

    #print("------ Final MSE Values -------")
    Y_pred = vander_matrix.dot(weights)
    getMSEOld(Y_pred, Y_values)

    Ein = getMSE(data_set, weights)
    Eout = getMSE(test_dataset, weights)
    return weights, Ein, Eout # this should be changed to the eventual estimated coefficients

def startingRandomCoeffs(d):
    """
    Get starting random coefficients, a0, a1, ..., ad, for the polynomial regression function
    :return: Randome coefficients for the polynomial based on random values between 0 and 1
    """
    return numpy.random.rand(d)

def experiment(N, d, variance):
    """
    Takes as input the size N of training dataset, the degree d of the model polynomial and noise variance σ^2.
    For given values, loops over M=50 trials, where each trial generates a dataset of size N with variance, σ^2, 
    and then fitting the data to a polynomial of degree d.

    :return: Output the average Ein_bar, Eout_bar which is the average Ein and Eout over M trials run after fitting the polynomial to the data
    """
    Ein_array = []
    Eout_array = []
    M = 50

    for i in range(M):
        training_dataset = getData(N, variance)
        test_dataset = getData(2000, variance)
        coeff,Ein_i,Eout_i = fitData(training_dataset, d, test_dataset)
        Ein_array.append(Ein_i)
        Eout_array.append(Eout_i)

    Ein_bar = numpy.mean(Ein_array)
    Eout_bar = numpy.mean(Eout_array)
    return Ein_bar, Eout_bar


In [12]:

#N = 2000      # Size of the training dataset
#variance = 0.2  # Variance, sigma^2
#degree_d=3    # Degree of the model polynomial

# data_set = getData(N, variance)
# test_dataset = getData(2000, variance)
# M = 35      # Number of trials
# est_coeff,Ein,Eout = fitData(data_set, degree_d, test_dataset)

N_values = [2, 5, 10, 20, 50, 100, 200]
d_values = [1, 2, 4, 8, 16, 32, 64]
variance_values = [0.05, 0.2]

for N in N_values:
    for d in d_values:
        for variance in variance_values:
            Ein_bar, Eout_bar = experiment(N, d, variance)


C:\Users\Nathan\AppData\Local\Temp\ipykernel_9492\187539522.py:56: RuntimeWarning: overflow encountered in square
  mse = numpy.mean((y_true - y_predicted)**2)
C:\Users\Nathan\AppData\Local\Temp\ipykernel_9492\187539522.py:38: RuntimeWarning: overflow encountered in square
  mse = numpy.mean((y_true_values - y_pred)**2)
C:\Users\Nathan\AppData\Local\Temp\ipykernel_9492\187539522.py:44: RuntimeWarning: overflow encountered in double_scalars
  squared_diff = (y_pred_i - y_true_i) ** 2
C:\Users\Nathan\AppData\Local\Temp\ipykernel_9492\187539522.py:90: RuntimeWarning: invalid value encountered in add
  weights += grad
